In [1]:
from __future__ import division, print_function,absolute_import

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import os

tf.__version__

import numpy as np
import pandas as pd


/home/michael/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:


def preprocess_data(data):
    x = data.iloc[:,1:].values/255
    y = data.label.values/255
    
    x = (x).astype('float32')
    y = (y).astype('int32')
    
    x = x.reshape(x.shape[0],28,28,1)
    y = keras.utils.to_categorical(y,num_classes=10)
    
    x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.1,random_state=42)
    
    return x_train, y_train, x_val,y_val


def cnn_model():
    
    model = keras.models.Sequential()
    
    model.add( keras.layers.Conv2D(32,(3,3),strides=(2,2),padding='same',activation='relu',input_shape=(256,256,3)))
    model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(keras.layers.Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(128,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Dense(256,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Dense(512,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Dense(1,activation = 'sigmoid'))
    
    model.compile(optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1),
                  loss='binary_crossentropy',metrics=['accuracy'])
    return model


model = cnn_model()
model.summary()


train_generator = ImageDataGenerator(featurewise_center=True,
                                     featurewise_std_normalization=True,
                                     rescale=1./255,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,vertical_flip=True)

val_generator = ImageDataGenerator(rescale=1./255)

train_data = train_generator.flow_from_directory(directory='dataset/training_set',class_mode='binary')
val_data = val_generator.flow_from_directory(directory='dataset/test_set',class_mode='binary')

model.fit_generator(train_data,steps_per_epoch=train_data.n,epochs=3,
                    validation_data=val_data,validation_steps=val_data.n)


FileNotFoundError: File b'train (3).csv' does not exist